# Test zur Erstellung einer Animation

Achtung: Der Code ist quick & dirty programmiert. Ich wollte nur testen, inwiefern man mit moviepy Lehrvideos erstellen kann. :-)

In [41]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import numpy as np

from moviepy.editor import *
from moviepy.video.io.bindings import mplfig_to_npimage

In [42]:
scenes = []
WIDTH = 480
HEIGHT = 270
WIDTH = 1920
HEIGHT = 1080
WHITE=(255,255,255)

In [43]:
import os
import csv

from IPython.core.display import display

class Speaker():
    
    def __init__(self, path):
        self.path = path
        self.audio = AudioFileClip(path)
        
        labels_file = os.path.join(os.path.dirname(path), "labels.txt")
        
        with open(labels_file, "r") as fd:
            self.sections = [ (float(x[0]), float(x[1])) for x in csv.reader(fd, delimiter="\t") ]
        
    def __len__(self):
        return len(self.sections)
        
    def __getitem__(self, key):
        start, end = self.sections[key]
        
        return self.audio.subclip(start, end)

class MovieFiles(object):
    def __init__(self, path="."):
        self.path = path
    
    def __getitem__(self, key):
        new_path = os.path.join(self.path, key)
        
        if os.path.isdir(new_path):
            return MovieFiles(new_path)
        elif os.path.isfile(new_path):
            if os.path.basename(new_path) == "speaker.flac":
                return Speaker(new_path)
        
        raise Exception(new_path)
        
data = MovieFiles()

In [44]:
def background(audio):
    b = CompositeVideoClip([ColorClip(size=(WIDTH,HEIGHT), color=(255,255,255), duration=audio.duration)])
    b.audio = audio
    return b

def add_to(l, e, n):
    assert len(l) > n-1
    
    if len(l) > n:
        l[n] = e
    else:
        l.append(e)

# Szene 1

1. Wenn wir uns stetige Funktionen anschauen, so können wir den Eindruck gewinnen, dass ihre Umkehrfunktionen immer stetig sind.
2. Nehmen wir zum Beispiel die Quadratfunktion auf dem Bereich der nicht negativen Zahlen.
3. Ihre Umkehrfunktion ist die Wurzelfunktion, welche wie die Quadratfunktion selbst stetig ist.
4. Ein weiteres Beispiel ist die Exponentialfunktion.
5. Sie ist stetig und ihre Umkehrfunktion, die Logarthmusfunktion, ist es auch.

In [45]:
scene = []
speaker = data["scenes"]["scene01"]["audio"]["speaker.flac"]

add_to(scene, background(speaker[0]), 0)

In [46]:
from skimage.color import rgba2rgb
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.collections import LineCollection

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis/np.sqrt(np.dot(axis, axis))
    a = np.cos(theta/2.0)
    b, c, d = -axis*np.sin(theta/2.0)
    aa, bb, cc, dd = a*a, b*b, c*c, d*d
    bc, ad, ac, ab, bd, cd = b*c, a*d, a*c, a*b, b*d, c*d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

def mpl_to_image(fig):
    canvas = FigureCanvasAgg(fig)
    canvas.draw()

    data = np.fromstring(canvas.tostring_argb(), dtype=np.uint8, sep='')
    data = data.reshape(canvas.get_width_height()[::-1] + (4,))
    data = np.roll(data, -1, 2)
    data = rgba2rgb(data) * 255

    return data

class PlotImage:
    
    w = WIDTH
    h = HEIGHT
    dpi = int(WIDTH * 72 / 480)
    
    def create_plot(self, fig):
        dpi = self.dpi
        fig.set_size_inches(self.w / dpi, self.h / dpi)
        
        ax = fig.add_subplot(111)
        #ax.patch.set_alpha(0)
        #ax.patch.set_facecolor('none')
        self.make_plot(ax)
    
    def show(self, t):
        self.t = t
        self.create_plot(plt.figure(dpi = self.dpi))
    
    def __call__(self, t):
        fig = Figure(dpi = self.dpi)
        self.t = t
        self.create_plot(fig)
        return mpl_to_image(fig)

class InverseFunctionPlot(PlotImage):

    phi = None
    func = None
    xlim = None
    ylim = None
    show_inv = True
    mirror_ticks = True
        
    def make_plot(self, ax):
        rot = rotation_matrix([1,1,0], self.phi)
        
        ax.set_xlim(self.xlim)
        
        if self.mirror_ticks:
            ax.set_yticks(ax.get_xticks())
        ax.set_ylim(self.ylim)

        x = np.linspace(*self.xlim,1000)
        y = self.func(x)
        #mask = (y >= self.ylim[0]) & (y <= self.ylim[1])
        
        #x = x[mask]
        #y = y[mask]
        
        if self.show_inv:
            ps = np.dot(rot, np.stack((x, y, np.zeros_like(x))))
            zs = ps[2][:-1]

            mask1 = zs < 0
            mask2 = zs >= 0

            lwidths=self.lw*self.zmax/(self.zmax - zs)

            points = np.array([ps[0], ps[1]]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)

            if sum(mask1) > 2:
                ax.add_collection(LineCollection(segments[mask1], linewidths=lwidths[mask1], color='blue', zorder=1))
        
        bx = self.black_graph_x
        if bx:
            ax.plot(bx, bx, color="black", lw=0.5)
        
        ax.plot(x, y, lw=self.lw)
        
        if self.show_inv and sum(mask2) > 2:
            ax.add_collection(LineCollection(segments[mask2], linewidths=lwidths[mask2], color='blue', zorder=10))

In [47]:
d1= speaker[1].duration
d2= speaker[2].duration
d2m = d2/5
d2r = (d2 - d2m)/2
        
class SquareFunc(InverseFunctionPlot):
    
    @property
    def show_inv(self):
        return self.t > d1+d2m
    
    @property
    def phi(self):
        t = self.t - d1 - d2m
        return -np.pi * min(t,d2r) / d2r
    
    @property
    def black_graph_x(self):
        if self.t <= d1:
            return None
        t = self.t-d1
        
        return (0,self.xlim[1] * min(t,d2m)/d2m)
        
    func = lambda self, x: x**2
    xlim = (0,3)
    
    @property
    def ylim(self):
        return (self.xlim[0] * self.h / self.w, self.xlim[1] * self.h / self.w)
    
    lw=3
    zmax = 1

v = VideoClip(SquareFunc(), duration=d1+d2)
v.audio = concatenate_audioclips([speaker[1],speaker[2]])
v = v.fadein(d1/4, initial_color=(255,255,255)).fadeout(d2/6, final_color=(255,255,255))

add_to(scene, v, 1)

In [48]:
d1= speaker[3].duration
d2= speaker[4].duration
d2m = d2/5
d2r = (d2 - d2m)/2
        
class ExpFunc(InverseFunctionPlot):
    mirror_ticks = False
    
    @property
    def show_inv(self):
        return self.t > d1+d2m
    
    @property
    def phi(self):
        t = self.t - d1 - d2m
        return -np.pi * min(t,d2r) / d2r
    
    @property
    def ylim(self):
        return (self.xlim[0] * self.h / self.w, self.xlim[1] * self.h / self.w)
    
    @property
    def black_graph_x(self):
        if self.t <= d1:
            return None
        t = self.t-d1
        
        return (0,4 * min(t,d2m)/d2m)
        
    func = lambda self, x: np.exp(x)
    xlim = (0,2)
    ylim = (0,3)
    
    lw=3
    zmax = 2

v = VideoClip(ExpFunc(), duration=d1+d2)
v.audio = concatenate_audioclips([speaker[3],speaker[4]])
v = v.fadein(d1/4, initial_color=(255,255,255)).fadeout(d2/6, final_color=(255,255,255))

add_to(scene, v, 2)

In [49]:
concatenate_videoclips(scene).ipython_display(fps=10)

100%|██████████| 235/235 [03:56<00:00,  1.55s/it]
